In [1]:
import pylab
import calendar
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime

In [2]:
raw_training_data = pd.read_csv("input/train.csv")
test_data = pd.read_csv("input/test.csv")

In [3]:
def clean_data(data):
    data['date'] = data.datetime.apply(lambda x: x.split()[0])
    data['hour'] = data.datetime.apply(lambda x: x.split()[1].split(':')[0]).astype('int')
    data['weekday'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').weekday())
    data['month'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').month)
    data = data.sample(frac=1)
    test_index = int(data.shape[0] * 0.8)
    training_label = data['count'][:test_index]
    validation_label = data['count'][test_index:]
    data = data.drop(['casual', 'datetime', 'date', 'registered', 'count', 'atemp'], axis=1)
    return data[:test_index], training_label, data[test_index:], validation_label

In [4]:
training_data, training_label, validate_data, validation_label = clean_data(raw_training_data)

In [7]:
training_data

,season,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month
6924,2,0,1,1,18.86,24,19.0012,20,4,4
5928,1,0,1,1,10.66,60,15.0013,5,4,2
7440,2,0,1,2,24.60,83,0.0000,9,2,5
6013,1,0,1,1,14.76,43,16.9979,19,0,2
5007,4,0,1,1,17.22,38,0.0000,17,4,12
...,...,...,...,...,...,...,...,...,...,...
5741,1,0,0,1,5.74,50,7.0015,9,5,1
4687,4,0,1,1,14.76,87,0.0000,9,1,11
4239,4,0,0,1,26.24,44,6.0032,16,5,10
7269,2,0,1,3,22.96,84,0.0000,6,2,5


In [17]:
def build_mode():
    return tf.keras.models.Sequential(layers=[
        tf.keras.layers.Dense(units=10, activation='relu', input_shape=(10,)),
        tf.keras.layers.Dense(units=1, activation='linear')
    ])

model = build_mode()

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss='mse', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(training_data, training_label, epochs=20, batch_size=128)

Epoch 1/20
8708/8708 [==============================] - 0s 19us/sample - loss: 50988.1930 - accuracy: 0.0075
Epoch 2/20
8708/8708 [==============================] - 0s 5us/sample - loss: 29264.8258 - accuracy: 0.0095
Epoch 3/20
8708/8708 [==============================] - 0s 6us/sample - loss: 24909.8790 - accuracy: 0.0095
Epoch 4/20
8708/8708 [==============================] - 0s 6us/sample - loss: 22584.7474 - accuracy: 0.0095
Epoch 5/20
8708/8708 [==============================] - 0s 6us/sample - loss: 21951.2250 - accuracy: 0.0095
Epoch 6/20
8708/8708 [==============================] - 0s 6us/sample - loss: 21730.0334 - accuracy: 0.0095
Epoch 7/20
8708/8708 [==============================] - 0s 5us/sample - loss: 21835.4058 - accuracy: 0.0095
Epoch 8/20
8708/8708 [==============================] - 0s 6us/sample - loss: 21555.3469 - accuracy: 0.0095
Epoch 9/20
8708/8708 [==============================] - 0s 5us/sample - loss: 21536.5914 - accuracy: 0.0095
Epoch 10/20
8708/8708 [====

The model does not learn anything. Scale the input.

In [10]:
from sklearn.preprocessing  import StandardScaler
def clean_data_scale(data):
    data['date'] = data.datetime.apply(lambda x: x.split()[0])
    data['hour'] = data.datetime.apply(lambda x: x.split()[1].split(':')[0]).astype('int')
    data['weekday'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').weekday())
    data['month'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').month)
    data = data.sample(frac=1)
    test_index = int(data.shape[0] * 0.8)
    training_label = data['count'][:test_index]
    validation_label = data['count'][test_index:]
    data = data.drop(['casual', 'datetime', 'date', 'registered', 'count', 'atemp'], axis=1)
    training_x = StandardScaler().fit_transform(data[:test_index])
    validation_x = StandardScaler().fit_transform(data[test_index:])
    return training_x, training_label, validation_x, validation_label

In [11]:
x_scale, training_label, validation_scale, validation_label = clean_data_scale(raw_training_data)

In [14]:
x_scale

array([[-0.46463149,  5.92729113, -1.4533489 , ..., -0.36532961,
         0.47999927, -0.74393055],
       [ 1.33519548, -0.16871113,  0.68806603, ..., -0.50996001,
        -0.01878756,  1.00267155],
       [ 1.33519548, -0.16871113, -1.4533489 , ..., -0.50996001,
         1.47757294,  1.2937719 ],
       ...,
       [ 1.33519548, -0.16871113,  0.68806603, ..., -0.65459041,
        -1.51514806,  1.2937719 ],
       [ 0.43528199,  5.92729113, -1.4533489 , ...,  0.21319199,
        -1.51514806,  0.7115712 ],
       [-0.46463149, -0.16871113,  0.68806603, ...,  1.08097438,
        -1.01636123, -0.16172985]])

In [18]:
history = model.fit(x_scale, training_label, epochs=20, batch_size=128)

Epoch 1/20
8708/8708 [==============================] - 0s 14us/sample - loss: 67460.4699 - accuracy: 0.0073
Epoch 2/20
8708/8708 [==============================] - 0s 5us/sample - loss: 55823.5133 - accuracy: 0.0094
Epoch 3/20
8708/8708 [==============================] - 0s 6us/sample - loss: 38163.3375 - accuracy: 0.0094
Epoch 4/20
8708/8708 [==============================] - 0s 6us/sample - loss: 27196.4404 - accuracy: 0.0094
Epoch 5/20
8708/8708 [==============================] - 0s 5us/sample - loss: 23634.7257 - accuracy: 0.0094
Epoch 6/20
8708/8708 [==============================] - 0s 5us/sample - loss: 22455.8890 - accuracy: 0.0094
Epoch 7/20
8708/8708 [==============================] - 0s 6us/sample - loss: 21936.3984 - accuracy: 0.0094
Epoch 8/20
8708/8708 [==============================] - 0s 6us/sample - loss: 21509.3442 - accuracy: 0.0094
Epoch 9/20
8708/8708 [==============================] - 0s 5us/sample - loss: 21098.3370 - accuracy: 0.0094
Epoch 10/20
8708/8708 [====

It does not help!